## Preprocessing

In [53]:
# Data Preparation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Modelling Metrics
from sklearn.metrics import classification_report, roc_auc_score

# Modelling
import mlflow
from mlflow.models.signature import infer_signature

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
import pytorch_lightning as pl

In [54]:
df = pd.read_csv("../data/tabular/binary-classification/titanic-dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [55]:
target_variable = "Survived"

In [56]:
df = pd.merge(
    left=df,
    right=pd.get_dummies(df["Sex"]),
    left_index=True,
    right_index=True,
    how="inner",
)
df = pd.merge(
    left=df,
    right=pd.get_dummies(df["Embarked"]),
    left_index=True,
    right_index=True,
    how="inner",
)

df["Cabin"] = df["Cabin"].str[0]
df = pd.merge(
    left=df,
    right=pd.get_dummies(df["Cabin"], prefix="Cabin"),
    left_index=True,
    right_index=True,
    how="inner",
)

df = pd.merge(
    left=df,
    right=pd.get_dummies(df["Embarked"], prefix="Embarked"),
    left_index=True,
    right_index=True,
    how="inner",
)

In [57]:
columns_to_drop = ["PassengerId", "Name", "Sex", "Ticket", "Fare", "Cabin", "Embarked"]

In [58]:
df.drop(columns_to_drop, axis=1, inplace=True)
df.dropna(inplace=True)

In [59]:
y = df[target_variable]
X = df.drop(target_variable, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101
)

# Training

In [60]:
def metrics(true_values: list, model_predictions: list, prefix: str = None) -> dict:

    classification_report_ = classification_report(
        y_true=true_values, y_pred=model_predictions, output_dict=True
    )
    roc = roc_auc_score(y_true=true_values, y_score=model_predictions)

    classification_report_["not_survived"] = classification_report_.pop("0")
    classification_report_["survived"] = classification_report_.pop("1")

    flattened_metrics = pd.json_normalize(classification_report_)
    flattened_metrics["roc_auc_score"] = roc

    if prefix:
        flattened_metrics.columns = [
            f"{prefix}_{column}" for column in flattened_metrics.columns
        ]

    flattened_metrics = flattened_metrics.to_dict(orient="records")[0]
    return flattened_metrics

In [61]:
mlflow_tracking_uri = "http://localhost:5000"
mlflow_experiment = "binary-classification"
model_name = "binary-classifier"

In [62]:
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment(mlflow_experiment)

2022/09/26 19:06:48 INFO mlflow.tracking.fluent: Experiment with name 'binary-classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/zbloss/Projects/mlflow-onnx-rust/mlruns/1', creation_time=1664233609003, experiment_id='1', last_update_time=1664233609003, lifecycle_stage='active', name='binary-classification', tags={}>

In [63]:
for model_class in [
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier,
    XGBClassifier,
]:
    with mlflow.start_run():

        model = model_class()
        model.fit(X_train, y_train)

        train_predictions = model.predict(X_train)
        test_predictions = model.predict(X_test)

        train_metrics = metrics(
            true_values=y_train, model_predictions=train_predictions, prefix="train"
        )
        test_metrics = metrics(
            true_values=y_test, model_predictions=test_predictions, prefix="test"
        )

        mlflow.log_metrics(metrics=train_metrics)
        mlflow.log_metrics(metrics=test_metrics)

        signature = infer_signature(X_test, test_predictions)
        
        if model_class == XGBClassifier:
            mlflow.xgboost.log_model(
                model,
                "model",
                signature=signature,
                input_example=X_test,
                registered_model_name=model_name,
            )
        else:
            mlflow.sklearn.log_model(
                model,
                "model",
                signature=signature,
                input_example=X_test,
                registered_model_name=model_name,
            )

/home/zbloss/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zbloss/.local/lib/python3.10/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing value

In [64]:
class TorchModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            *[
                nn.Linear(X_train.shape[-1], X_train.shape[-1] * 2),
                nn.ReLU(),
                nn.Linear(X_train.shape[-1] * 2, X_train.shape[-1] * 2),
                nn.ReLU(),
                nn.Linear(X_train.shape[-1] * 2, X_train.shape[-1]),
                nn.ReLU(),
                nn.Linear(X_train.shape[-1], 1),
                nn.Sigmoid(),
            ]
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.float()
        y = y.unsqueeze(-1).float()

        prediction = self(x)

        loss = F.binary_cross_entropy(prediction, y)
        return loss

    def validation_step(self, batch, batch_idx):

        x, y = batch
        x = x.float()
        y = y.unsqueeze(-1).float()

        prediction = self(x)
        loss = F.binary_cross_entropy(prediction, y)

    def test_step(self, batch, batch_idx):

        x, y = batch
        x = x.float()
        y = y.unsqueeze(-1).float()

        prediction = self(x)
        loss = F.binary_cross_entropy(prediction, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


model = TorchModel()

In [65]:
class TitanicDataset(Dataset):
    def __init__(self, target_variable="Survived"):
        import pandas as pd

        df = pd.read_csv("../data/tabular/binary-classification/titanic-dataset.csv")
        df = pd.merge(
            left=df,
            right=pd.get_dummies(df["Sex"]),
            left_index=True,
            right_index=True,
            how="inner",
        )
        df = pd.merge(
            left=df,
            right=pd.get_dummies(df["Embarked"]),
            left_index=True,
            right_index=True,
            how="inner",
        )

        df["Cabin"] = df["Cabin"].str[0]
        df = pd.merge(
            left=df,
            right=pd.get_dummies(df["Cabin"], prefix="Cabin"),
            left_index=True,
            right_index=True,
            how="inner",
        )

        df = pd.merge(
            left=df,
            right=pd.get_dummies(df["Embarked"], prefix="Embarked"),
            left_index=True,
            right_index=True,
            how="inner",
        )
        columns_to_drop = [
            "PassengerId",
            "Name",
            "Sex",
            "Ticket",
            "Fare",
            "Cabin",
            "Embarked",
        ]
        df.drop(columns_to_drop, axis=1, inplace=True)
        df.dropna(inplace=True)

        self.y = df[target_variable].to_numpy()
        self.X = df.drop(target_variable, axis=1).to_numpy().astype(float)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [66]:
data = TitanicDataset()
dataloader = DataLoader(data, batch_size=4)

In [67]:
test_size = 0.2
val_size = 0.2

test_size *= len(data)
val_size *= len(data)
test_size = int(test_size)
val_size = int(val_size)

train_size = len(data) - test_size - val_size

train, test, val = random_split(data, (train_size, test_size, val_size))


batch_size = 4
train_loader = DataLoader(train, batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size, shuffle=False)
val_loader = DataLoader(val, batch_size, shuffle=True)

In [68]:
trainer = pl.Trainer(
    logger=pl.loggers.MLFlowLogger(
        experiment_name=mlflow_experiment,
        tracking_uri=mlflow_tracking_uri,
    ),
    max_epochs=20,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [69]:
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 3.3 K 
-------------------------------------
3.3 K     Trainable params
0         Non-trainable params
3.3 K     Total params
0.013     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/zbloss/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:489: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/zbloss/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/zbloss/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [70]:
m = trainer.test(model, test_loader)

/home/zbloss/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

In [71]:
with mlflow.start_run():

    preds = [model(torch.tensor(x).float()) for x, y in train]
    train_predictions = (
        (torch.stack(preds).reshape(-1) > 0.5).detach().numpy().astype(int)
    )

    preds = [model(torch.tensor(x).float()) for x, y in test]
    test_predictions = (
        (torch.stack(preds).reshape(-1) > 0.5).detach().numpy().astype(int)
    )

    train_metrics = metrics(
        true_values=[y for x, y in train],
        model_predictions=train_predictions,
        prefix="train",
    )
    test_metrics = metrics(
        true_values=[y for x, y in test],
        model_predictions=test_predictions,
        prefix="test",
    )

    mlflow.log_metrics(metrics=train_metrics)
    mlflow.log_metrics(metrics=test_metrics)

    x = train[0][0]
    signature = infer_signature(x, model(torch.tensor(x).float()).detach().numpy())
    mlflow.pytorch.log_model(
        model,
        "model",
        signature=signature,
        input_example=X_test,
        registered_model_name=model_name,
    )

Registered model 'binary-classifier' already exists. Creating a new version of this model...
2022/09/26 19:07:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: binary-classifier, version 5
Created version '5' of model 'binary-classifier'.


## Load and Convert to ONNX

In [77]:
import onnx
from onnx import helper as h
from onnx import TensorProto as tp
from onnx import checker
from onnx import save

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

import onnxruntime
from skl2onnx import __max_supported_opset__

import json
import onnxmltools
from onnxconverter_common.data_types import FloatTensorType

In [78]:
sample_input = X_train.to_numpy()[0].reshape(1, -1).astype(np.float32)

In [80]:
model_version = 4
model_uri = f"models:/{model_name}/{model_version}"
mlflow_model = mlflow.pyfunc.load_model(model_uri=model_uri)
print(mlflow_model.metadata.to_yaml())
ml_flavor = mlflow_model.metadata.to_dict()["flavors"]["python_function"][
    "loader_module"
]

if ml_flavor == "mlflow.sklearn":
    model = mlflow.sklearn.load_model(model_uri)
    prediction = model.predict(sample_input)

if ml_flavor == "mlflow.pytorch":
    model = mlflow.pytorch.load_model(model_uri)
    prediction = model(torch.tensor(sample_input).float())
    
if ml_flavor == "mlflow.xgboost":
    model = mlflow.xgboost.load_model(model_uri)
    prediction = model.predict(sample_input)

prediction

artifact_path: model
flavors:
  python_function:
    data: model.xgb
    env: conda.yaml
    loader_module: mlflow.xgboost
    python_version: 3.10.7
  xgboost:
    code: null
    data: model.xgb
    model_class: xgboost.sklearn.XGBClassifier
    xgb_version: 1.6.2
mlflow_version: 1.29.0
model_uuid: 37cecb1012894c9c92ea89537ce1ca73
run_id: c5af0f3cd17d4f9b9639e6b2e72dc20c
saved_input_example_info:
  artifact_path: input_example.json
  pandas_orient: split
  type: dataframe
signature:
  inputs: '[{"name": "Pclass", "type": "long"}, {"name": "Age", "type": "double"},
    {"name": "SibSp", "type": "long"}, {"name": "Parch", "type": "long"}, {"name":
    "female", "type": "integer"}, {"name": "male", "type": "integer"}, {"name": "C",
    "type": "integer"}, {"name": "Q", "type": "integer"}, {"name": "S", "type": "integer"},
    {"name": "Cabin_A", "type": "integer"}, {"name": "Cabin_B", "type": "integer"},
    {"name": "Cabin_C", "type": "integer"}, {"name": "Cabin_D", "type": "integer"},


array([0])

In [86]:
number_of_features = np.array(mlflow.artifacts.load_dict(f'{model_uri}/input_example.json')['data']).shape[-1]
initial_type = [("float_input", FloatTensorType([None, number_of_features]))]

In [88]:
torch.rand((1, number_of_features)).numpy().tolist()

[[0.09830367565155029,
  0.9001232981681824,
  0.012955546379089355,
  0.8799967765808105,
  0.05108916759490967,
  0.6351413726806641,
  0.28915196657180786,
  0.8558712601661682,
  0.3940802216529846,
  0.6569401621818542,
  0.09095829725265503,
  0.11667466163635254,
  0.7198678255081177,
  0.6807432770729065,
  0.5608528256416321,
  0.7539402842521667,
  0.7674431204795837,
  0.9450421333312988,
  0.17658114433288574,
  0.5480032563209534]]

In [ ]:
onnxmltools.convert.convert_sklearn()

In [81]:
if ml_flavor == "mlflow.sklearn":
    initial_type = [("float_input", FloatTensorType([None, sample_input.shape[-1]]))]
    onx = convert_sklearn(model, initial_types=initial_type)
    with open("model.onnx", "wb") as f:
        f.write(onx.SerializeToString())
        f.close()

if ml_flavor == "mlflow.pytorch":
    # Export the model
    torch.onnx.export(
        model,  # model being run
        torch.tensor(
            sample_input
        ).float(),  # model input (or a tuple for multiple inputs)
        "model.onnx",  # where to save the model (can be a file or file-like object)
        export_params=True,  # store the trained parameter weights inside the model file
        do_constant_folding=True,  # whether to execute constant folding for optimization
        input_names=["float_input"],  # the model's input names
        output_names=["output_label"],  # the model's output names
        dynamic_axes={
            "float_input": {0: "batch_size"},  # variable length axes
            "output_label": {0: "batch_size"},
        },
    )

In [95]:
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

In [96]:
session = onnxruntime.InferenceSession("model.onnx")
input_name = session.get_inputs()[0].name
label_name = session.get_outputs()[0].name

input_name, label_name

('float_input', 'output_label')

In [97]:
pred_onx = session.run([label_name], {input_name: sample_input})[0]
pred_onx

array([[0.25795278]], dtype=float32)

In [99]:
sample_input

array([[ 1., 28.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32)

## Debug Endpoint

In [46]:
import requests

base_url = "http://localhost:8000"
endpoint = "predict"

In [50]:
payload = {"data": sample_input.tolist()}

In [132]:
payload

{'data': [[1.0,
   28.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0]]}

In [154]:
response = requests.post(
    url=f"{base_url}/{endpoint}",
    headers={"Content-Type": "application/json", "accept": "application/json"},
    data=json.dumps(payload),
)
response.status_code
prediction = response.json()["prediction"]
prediction

[1]

In [157]:
response = requests.post(
    url=f"{base_url}/predict-without-onnx",
    headers={"Content-Type": "application/json", "accept": "application/json"},
    data=json.dumps(payload),
)
response.status_code
prediction = response.json()["prediction"]
prediction

1